In [2]:
import json
import pandas as pd
from transformers import pipeline

In [3]:
with open('/content/test-A%2FSPIQA_testA.json', 'r') as file:
    data = json.load(file)

In [4]:
rows = []
for paper, content in data.items():
    for qa in content['qa']:
        rows.append({
            'paper': paper,
            'question': qa['question'],
            'answer': qa['answer'],
            'reference_figure': qa['reference'],
            'reference_figure_caption': content['all_figures'][qa['reference']]['caption']
        })

df = pd.DataFrame(rows)


In [5]:
t5_pipeline = pipeline("text2text-generation", model="t5-small")

def generate_answer_t5(question, context):
    prompt = f"question: {question} context: {context}"
    output = t5_pipeline(prompt, max_length=50)[0]['generated_text']
    return output.strip()


df['generated_answer'] = df.apply(lambda row: generate_answer_t5(row['question'], row['reference_figure_caption']), axis=1)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [6]:
df.to_csv("t5_results.csv", index=False)